# ENSO

This notebook provides an overview of running the ENSO metrics. More information can be found in the [README]( ). Example parameter files are located in the [PMP sample setups]( ). 

**Reference**: Planton, Y., E. Guilyardi, A. T. Wittenberg, J. Lee, P. J. Gleckler, T. Bayr, S. McGregor, M. J. McPhaden, S. Power, R. Roehrig, A. Voldoire, 2020: Evaluating El Niño in climate models with the CLIVAR 2020 ENSO metrics package. Bulletin of the American Meteorological Society. [doi: 10.1175/BAMS-D-19-0337.1](https://doi.org/10.1175/BAMS-D-19-0337.1)

Description for individual metrics can be found at https://github.com/CLIVAR-PRP/ENSO_metrics/wiki.

### Requirements

The first two sections of this notebook help to set up the demo data and environment.

**Download demo data**: The ENSO metrics demo requires downloading a large sample data set (size 10.8 GB). Some files overlap with the PMP sample data from Demo 0 and will not be downloaded twice if that data is already present.

**Environment**: This demo also requires installing the ENSO Metrics and scipy packages to your current environment. *If you do not want to alter your PCMDI Metrics conda environment*, it is suggested that you create and activate a clone for use with this demo.

## Download demo data

The ENSO metric requires a different set of sample data than the rest of the PMP metrics. This section of the notebook will download that data to your chosen location and generate a basic parameter file.

In [1]:
# Lets get the file containing the data needed for this demo
import requests
r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.txt")
with open("enso_data_files.txt","wb") as f:
    f.write(r.content)

If you want to change the location where the demo data and output are stored, you can do so here:

In [2]:
# This is where you will be downloading the sample_data
demo_data_directory = "demo_data_enso_new_obs"
# this line is where your output will be stored
demo_output_directory = "demo_output_enso_new_obs"

Then download the data. The total sample data size is 10.8 GB. This will take several minutes.

In [3]:
# Let's download the files
import cdat_info
try:
    cdat_info.download_sample_data_files("enso_data_files.txt", demo_data_directory)
    print("All files downloaded")
except:
    print("Download failed")

MD5: enso_data_files.txt
All files downloaded


After downloading the data, we generate the parameter file for this demo.

In [4]:
from download_sample_data import generate_parameter_files
filenames=["basic_enso_param.py.in"]
generate_parameter_files(demo_data_directory, demo_output_directory, filenames=filenames)

Preparing parameter file: basic_enso_param.py
Saving User Choices


## Environment

[ENSO Metrics package](https://github.com/CLIVAR-PRP/ENSO_metrics) and [scipy](https://www.scipy.org/) installations are needed. This section will clone the ENSO Metrics repository and *install ENSO Metrics and scipy in your current conda environment*. Set the `enso_install_location` below to chose where to clone the ENSO Metrics repository.

In [5]:
enso_install_location = "."

To clone and install the ENSO Metrics package, un-comment the next cell and run it (delete quotes in lines 1 & 6).

In [6]:
"""
%%bash -s "$enso_install_location"
cd $1
git clone https://github.com/CLIVAR-PRP/ENSO_metrics.git
cd ENSO_metrics
python setup.py install
"""

'\n%%bash -s "$enso_install_location"\ncd $1\ngit clone https://github.com/CLIVAR-PRP/ENSO_metrics.git\ncd ENSO_metrics\npython setup.py install\n'

To install scipy, un-comment the next cell and run it (delete quotes in lines 1 & 3).

In [7]:
"""
conda install -c anaconda scipy 
"""

'\nconda install -c anaconda scipy \n'

## Usage

The ENSO driver can be run from the command line as `enso_driver.py`. In this notebook, we will use bash cell magic (cells beginning with `%%bash`) to run the ENSO driver as a subprocess.

For help, type:  
```
enso_driver.py --help
```

In [8]:
%%bash
enso_driver.py --help

usage: enso_driver.py [-h] [--parameters PARAMETERS]
                      [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                      [--mip MIP] [--exp EXP] [--modpath MODPATH]
                      [--modpath_lf MODPATH_LF]
                      [--modnames MODNAMES [MODNAMES ...]] [-r REALIZATION]
                      [--reference_data_path REFERENCE_DATA_PATH]
                      [--reference_data_lf_path REFERENCE_DATA_LF_PATH]
                      [--metricsCollection METRICSCOLLECTION]
                      [--json_name JSON_NAME] [--netcdf_name NETCDF_NAME]
                      [--results_dir RESULTS_DIR] [--case_id CASE_ID]
                      [--obs_catalogue OBS_CATALOGUE]
                      [--obs_cmor_path OBS_CMOR_PATH] [-d [DEBUG]]
                      [--obs_cmor [OBS_CMOR]] [--nc_out [NC_OUT]]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PA

### Basic example

Parameters for the ENSO Metrics can be set on the command line or using a parameter file. This first example will use a parameter file, which is shown below.

In [9]:
with open("basic_enso_param.py") as f:
    print(f.read())

import os

#
#  OPTIONS ARE SET BY USER IN THIS FILE AS INDICATED BELOW BY:
#
#

# MODELS
modnames = ['ACCESS1-0']
mip = 'cmip5'  # cmip5, cmip6
exp = 'historical'  # historical, piControl
realization = 'r1i1p1'
modpath = 'demo_data_enso_new_obs/CMIP5_demo_data/%(variable)_Amon_%(model)_historical_%(realization)_185001-200512.nc'
modpath_lf = 'demo_data_enso_new_obs/CMIP5_demo_data/sftlf_fx_%(model)_amip_r0i0p0.nc'

# OBSERVATIONS
obs_cmor = True
obs_cmor_path = "demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly"
obs_catalogue = "demo_data/obs4MIPs_PCMDI-CEM2021_monthly_bySource_catalogue_v20210805_demo.json"

# METRICS COLLECTION
metricsCollection = 'ENSO_perf'  # ENSO_perf, ENSO_tel, ENSO_proc

# OUTPUT
case_id = 'basicTestEnso'
results_dir = os.path.join('demo_output_enso_new_obs',case_id, metricsCollection)

json_name = '%(mip)_%(exp)_%(metricsCollection)_%(case_id)_%(model)_%(realization)'
netcdf_name = json_name
nc_out = False



The next cell runs the ENSO driver using the basic parameter file. This may take several minutes.

In [10]:
%%bash
enso_driver.py -p basic_enso_param.py

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: demo_output_enso_new_obs/basicTestEnso/ENSO_perf
netcdf_path: demo_output_enso_new_obs/basicTestEnso/ENSO_perf
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_796_g28c5a3f4-py3.8.egg-tmp/share/pmp
output directory for graphics:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
output directory for diagnostic_results:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
output directory for metrics_results:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
list_variables: ['pr', 'sst', 'taux']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
Process start: Fri Aug 27 14:36:00 2021
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- va

/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-08-27 14:43::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-08-27 14:43::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


This run saved metrics to two files:  
basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json   basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json

diveDown metrics are not available in all cases. 

The results section of cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json is shown below.

In [11]:
import json
metrics_file=demo_output_directory+"/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json"
with open(metrics_file) as f:
    results = json.load(f)["RESULTS"]["model"]["ACCESS1-0"]["r1i1p1"]["value"]
print(json.dumps(results, indent = 2))

{
  "BiasPrLatRmse": {
    "diagnostic": {
      "ACCESS1-0_r1i1p1": {
        "value": null,
        "value_error": null
      },
      "ERA-Interim": {
        "value": null,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": null,
        "value_error": null
      }
    },
    "metric": {
      "ERA-Interim": {
        "value": 1.107530462384606,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": 1.9301242972172135,
        "value_error": null
      }
    }
  },
  "BiasPrLonRmse": {
    "diagnostic": {
      "ACCESS1-0_r1i1p1": {
        "value": null,
        "value_error": null
      },
      "ERA-Interim": {
        "value": null,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": null,
        "value_error": null
      }
    },
    "metric": {
      "ERA-Interim": {
        "value": 0.6464917771721342,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": 1.4165839641155153,
        "value_e

### ENSO Metrics Collections

There are 3 metrics collections available:  
ENSO_perf  
ENSO_tel  
ENSO_proc  

They can be selected using the `--metricsCollection` flag. The first example used the "ENSO_perf" collection.

The next example runs the teleconnection collection. To save individual metrics in netCDF format, it uses the `--nc_out` flag.

In [12]:
%%bash -s "$demo_output_directory"
enso_driver.py -p basic_enso_param.py \
--metricsCollection ENSO_tel \
--results_dir $1/basicTestEnso/ENSO_tel \
--nc_out True

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_tel
outdir: demo_output_enso_new_obs/basicTestEnso/ENSO_tel
netcdf_path: demo_output_enso_new_obs/basicTestEnso/ENSO_tel
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_796_g28c5a3f4-py3.8.egg-tmp/share/pmp
output directory for graphics:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
output directory for diagnostic_results:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
output directory for metrics_results:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
list_variables: ['pr', 'sst']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
Process start: Fri Aug 27 14:58:35 2021
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
va

/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-08-27 15:09::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-08-27 15:09::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


All of the results (netCDF and JSON) are located in the output directory, which uses the metrics collection name.

In [13]:
!ls {demo_output_directory + "/basicTestEnso/ENSO_tel/*.nc"}

demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoAmpl.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoPrMapDjf.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoPrMapJja.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSeasonality.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstLonRmse.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstMapDjf.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstMapJja.nc


Finally, this example runs the remaining metrics collection ENSO_proc:

In [14]:
%%bash -s "$demo_output_directory"
enso_driver.py -p basic_enso_param.py \
--metricsCollection ENSO_proc \
--results_dir $1/basicTestEnso/ENSO_proc

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_proc
outdir: demo_output/basicTestEnso/ENSO_proc
netcdf_path: demo_output/basicTestEnso/ENSO_proc
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_796_g28c5a3f4-py3.8.egg-tmp/share/pmp
output directory for graphics:demo_output/basicTestEnso/ENSO_proc
output directory for diagnostic_results:demo_output/basicTestEnso/ENSO_proc
output directory for metrics_results:demo_output/basicTestEnso/ENSO_proc
list_variables: ['ssh', 'sst', 'taux', 'thf']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
Observation dataset AVISO-1-0 is not given for variable thf
Observation dataset GPCP-2-3 is not given for variable thf
Observation dataset HadISST-1-1 is not given for variable thf
PMPdriver: dict_obs readin end
Process start: Fri Aug 27 15:09:38 2021
models: ['ACCESS1-0']
 ----- model:  ACCESS

/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-08-27 15:14::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/basicTestEnso/ENSO_proc/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-08-27 15:15::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/basicTestEnso/ENSO_proc/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json
